# Création d'une base de donnée équilibrée

In [1]:
import pandas as pds
import numpy as np
import matplotlib.pyplot as plt
import random

Après des premiers test avec notre autoencodeur, nous nous sommes rendus compte de plusieurs chose. La base de données est composées de plus de femmes que d'hommes. Cette composition implique que notre autoencodeur reconnaissait des femmes même quand l'image d'origine était un homme.
Pour résoudre ce problème et éviter que ce problème se répètes avec d'autre attributs nous avons créer une bases de données ou chaque attributs est représenté au moins un certain nombre de fois et le nombre d'hommes et de femmes est sur tout les attributs le même.
Nous avons aussi séparer la base de données en plus petits dataset de manière à ce que l'entrainement puissent se faire plus facilement.

In [2]:
attributs_images = pds.read_csv("new_list_attr_celba.csv", sep=",", low_memory=False)

In [3]:
def compte_homme_femme(dataframe):
    """Permet de compter le nombre d'hommes et de femmes dans le dataset
    Parameters:
        dataframe (pandas.Dataframe): Dataframe contenant les différents attributs par images avec 1 quand l'attribut est présent et -1 quand il ne l'est pas
    Return:
        nombre_hommes (int): le nombre d'homme dans le dataset
        nombre_femmes (int) : le nombre de femme dans le dataset
    """
    nombre_hommes=0
    nombre_femmes=0

    for num_image in dataframe.index:
            if dataframe['Male'][num_image] == 1:
                nombre_hommes += 1
            elif dataframe['Male'][num_image] == -1:
                nombre_femmes += 1

    return nombre_hommes, nombre_femmes


In [4]:
nombre_hommes, nombre_femmes =  compte_homme_femme(attributs_images)
print("Nombres d'hommes : ", nombre_hommes, "\nNombres de femmes : ", nombre_femmes)

Nombres d'hommes :  84434 
Nombres de femmes :  118165


On regarde ensuite la représentation des hommes et des femmes pour chaque attributs.

In [5]:
def compte_attributs_par_sexe(dataframe, nombre_hommes, nombre_femmes): 
    """Permet de compter le nombre d'hommes et de femmes pour chaque attributs et print les resultats et renvois les attributs qui ont peu de representant
    Parameters:
        dataframe (pandas.Dataframe): Dataframe contenant les différents attributs par images avec 1 quand l'attribut est présent et -1 quand il ne l'est pas
        nombre_hommes (int) : nombre d'hommes dans tout le dataset
        nombre_femmes (int) : nombre de femmes dans tout le dataset
    Return:
        liste_tuples_attributs_nombres(list) : liste de tuples contenant les attributs et une liste du nombre d'hommes et de femmes pour cet attribut
    """
    total_individus = nombre_hommes + nombre_femmes
    liste_attributs = dataframe.columns
    liste_tuples_attributs_nombres = []

    for numero_attributs in range(1,len(liste_attributs)):
        attribut = liste_attributs[numero_attributs]
        compteur_hommes = 0
        compteur_femmes = 0
        total_compteur = 0
        for num_image in dataframe.index:
            if dataframe['Male'][num_image]==1 and dataframe[attribut][num_image]==1:
                compteur_hommes += 1
            elif dataframe['Male'][num_image]==-1 and dataframe[attribut][num_image]==1:
                compteur_femmes += 1
        total_compteur = compteur_hommes + compteur_femmes
        
        print(attribut," :","\n \t nombre d'hommes: ", compteur_hommes,"  ",round((compteur_hommes/total_compteur*100),1), "%\n \t nombre de femmes: ",compteur_femmes,"  ",round((compteur_femmes/total_compteur*100),1),"%")
        print("pourcentage de l'attribut dans le dataset : ",round((total_compteur/total_individus*100),2),"%")
        
        attributs_et_nombres = (attribut, [compteur_hommes, compteur_femmes])
        liste_tuples_attributs_nombres.append(attributs_et_nombres)
    
    return liste_tuples_attributs_nombres

In [6]:
liste_tuples_attributs_nombres = compte_attributs_par_sexe(attributs_images, nombre_hommes, nombre_femmes)

5_o_Clock_Shadow  : 
 	 nombre d'hommes:  22496    99.9 %
 	 nombre de femmes:  20    0.1 %
pourcentage de l'attribut dans le dataset :  11.11 %
Arched_Eyebrows  : 
 	 nombre d'hommes:  4513    8.3 %
 	 nombre de femmes:  49577    91.7 %
pourcentage de l'attribut dans le dataset :  26.7 %
Attractive  : 
 	 nombre d'hommes:  23579    22.7 %
 	 nombre de femmes:  80254    77.3 %
pourcentage de l'attribut dans le dataset :  51.25 %
Bags_Under_Eyes  : 
 	 nombre d'hommes:  29404    70.9 %
 	 nombre de femmes:  12042    29.1 %
pourcentage de l'attribut dans le dataset :  20.46 %
Bald  : 
 	 nombre d'hommes:  4530    99.6 %
 	 nombre de femmes:  17    0.4 %
pourcentage de l'attribut dans le dataset :  2.24 %
Bangs  : 
 	 nombre d'hommes:  6950    22.6 %
 	 nombre de femmes:  23759    77.4 %
pourcentage de l'attribut dans le dataset :  15.16 %
Big_Lips  : 
 	 nombre d'hommes:  13179    27.0 %
 	 nombre de femmes:  35606    73.0 %
pourcentage de l'attribut dans le dataset :  24.08 %
Big_Nose  

Avec ces informations, un choix des attributs et du nombre de fois où il serait représenté a été fait à la main. Il pourrait être automatisé mais étant donnée qu'il y a un plus grand nombre d'attributs où les hommes sont plus représentés que les femmes, nous avons écarté certains attributs vestimentaires, mais d'autres attributs physiques sont très peu représentés ce qui compliquait l'automatisation en gardant le même nombre d'hommes et de femmes.

La fonction *compte_attributs_par_sexe* renvoie cependant une liste de tuples contenant le nom de l'attribut et une liste avec le nombre d'hommes et le nombre de femmes dans le cas où une fonction automatisant la sélection voudrait être développée dans le futur.

In [ ]:
def ajout_images_par_attributs(dataframe_complet, nouveau_dataframe_reduit ,liste_attributs, nombre_a_ajoute, mode):
    """Permet d'ajouter le nombre d'images d'hommes et de femmes avec l'attribut à une liste
    Parameters:
        dataframe_complet (pandas.Dataframe): Dataframe contenant les différents attributs par images avec 1 quand l'attribut est présent et -1 quand il ne l'est pas
        nouveau_dataframe_reduit (pandas.Dataframe) : Dataframe avec les images selectionnées
        liste_attributs (list) : liste avec les attributs à ajouter
        nombre_a_ajoute (int) : nombre d'image a joute par attributs et par sexe
        mode (string) : "deux" quand il faut ajouter dans les deux sexe, "femme" quand on ajoute que des femmes pour l'attribut, "homme" quand on ajoute que des hommes pour l'attribut    
    Return:
        dataframe_complet (pandas.Dataframe): Dataframe donné en entré avec en moins les lignes des images ajouté dans la liste_numero_images
        nouveau_dataframe_reduit (pandas.Dataframe) : Dataframe avec les nouvelles images selectionnées ajouté
    """
    
    if mode=='deux':    
        for attribut in liste_attributs :
            compteur_hommes = 0
            compteur_femmes = 0
            for num_image in dataframe_complet.index:
                
                if compteur_hommes == nombre_a_ajoute and compteur_femmes == nombre_a_ajoute : 
                    break
                
                if dataframe_complet['Male'][num_image] == 1 and dataframe_complet[attribut][num_image] == 1 and compteur_hommes < nombre_a_ajoute:
                    nouveau_dataframe_reduit= nouveau_dataframe_reduit.append(dataframe_complet.loc[num_image])
                    dataframe_complet = dataframe_complet.drop([num_image], axis="index")
                    compteur_hommes += 1
                elif dataframe_complet['Male'][num_image] == -1 and dataframe_complet[attribut][num_image] == 1 and compteur_femmes < nombre_a_ajoute:
                    nouveau_dataframe_reduit= nouveau_dataframe_reduit.append(dataframe_complet.loc[num_image])
                    dataframe_complet = dataframe_complet.drop([num_image], axis="index")
                    compteur_femmes += 1
        return dataframe_complet, nouveau_dataframe_reduit
            
    elif mode == 'femme' or mode == 'homme':
        sexe = 1
        if mode == 'femme':
            sexe = -1
        for attribut in liste_attributs :
            compteur = 0
            for num_image in dataframe_complet.index:
                if dataframe_complet['Male'][num_image] == sexe and dataframe_complet[attribut][num_image]==1 and compteur<nombre_a_ajoute:
                    nouveau_dataframe_reduit= nouveau_dataframe_reduit.append(dataframe_complet.loc[num_image])
                    compteur += 1
                    dataframe_complet = dataframe_complet.drop([num_image], axis="index")
                if compteur == nombre_a_ajoute:
                    break
        return dataframe_complet, nouveau_dataframe_reduit
 

In [ ]:
liste_attributs_hommes_1000 = ["5_o_Clock_Shadow", "Goatee", "Sideburns", "Wearing_Necktie"]
liste_attributs_hommes_2000 = ["Bald", "Mustache"]
liste_attributs_femmes = ["Heavy_Makeup", "Rosy_Cheeks", "Wearing_Earrings", "Wearing_Lipstick"]
liste_deux_1000 = ["Blond_Hair",  "Gray_Hair", "Pale_Skin", "Arched_Eyebrows","Wearing_Hat", "Attractive", "Bags_Under_Eyes", "Bangs" ,"Big_Lips", "Big_Nose" , "Black_Hair", "Blurry", "Brown_Hair", "Bushy_Eyebrows", "Eyeglasses", "High_Cheekbones", "Mouth_Slightly_Open", "Narrow_Eyes", "No_Beard", "Oval_Face", "Pointy_Nose", "Receding_Hairline", "Smiling", "Straight_Hair", "Wavy_Hair", "Young"]
liste_deux_500 = ["Chubby" ,"Double_Chin"]

nouveau_dataframe_reduit= pds.DataFrame()

attributs_images, nouveau_dataframe_reduit = ajout_images_par_attributs(attributs_images, nouveau_dataframe_reduit ,liste_deux_500, 500, "deux")
attributs_images, nouveau_dataframe_reduit = ajout_images_par_attributs(attributs_images, nouveau_dataframe_reduit ,liste_deux_1000, 1000, "deux")
attributs_images, nouveau_dataframe_reduit = ajout_images_par_attributs(attributs_images, nouveau_dataframe_reduit ,liste_attributs_hommes_1000, 1000, "homme")
attributs_images, nouveau_dataframe_reduit = ajout_images_par_attributs(attributs_images, nouveau_dataframe_reduit ,liste_attributs_hommes_2000, 2000, "homme")
attributs_images, nouveau_dataframe_reduit = ajout_images_par_attributs(attributs_images, nouveau_dataframe_reduit ,liste_attributs_femmes, 2000, "femme")


In [ ]:
nouveau_dataframe_reduit.shape

In [ ]:
nouveau_dataframe_reduit.head

On melange ensuite la liste et on divise ensuite le dataset de 70000 images équilibré en des dataset plus petit de 1000 images afin de pouvoir entrainer notre modèle en plusieurs fois. En effet, en fonction de l'ordinateur utilisé, un trop grand nombre d'images n'est pas gerable pour l'ordinateur et 1000 images semble pouvoir être gerable pour différente puissance d'ordinateur. 

In [ ]:
def  creation_petit_dataset(liste_numero_images, nombre_par_dataset, chemin_sauvegarde) :
    """Permet la creation de plus petit dataset pour l'entrainement du modèle
    Parameters:
        liste_numero_images (list) : liste avec les images selectionnées sur lesquels le modèle sera entrainé
        nombre_par_dataset (int) : nombre d'images par petit dataset
        chemin_sauvegarde (string) : chemin pour la sauvegarde des nouveaux dataset
    Return:
        None
    """
    liste_numero_images= liste_numero_images['nb_pic']
    liste_numero_images= liste_numero_images.sort_values(0) 
    liste_data = [liste_numero_images[i:i+1000] for i in range(0,len(liste_numero_images),nombre_par_dataset)]
    for i in range(len(liste_data)):
        filename=chemin_sauvegarde+ "/dataset"+str(i)+".txt"
        liste_data[i].to_csv(filename, header=None, index=None, sep='\t', mode='a')
    

In [ ]:
creation_petit_dataset(nouveau_dataframe_reduit, 1000, "./small_dataset")

# Création de l'encodeur et du décodeur 

In [ ]:
import os
from PIL import Image
import keras
import tensorflow as tf
from tensorflow.keras import layers,metrics
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
import matplotlib.pyplot as plt
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D
from keras.models import Model
from sklearn.model_selection import train_test_split
from keras.models import Model, load_model

In [ ]:
def load_dataset(chemin_vers_images, chemin_vers_datalist, largeur, hauteur) :
    """Charge les images dont le nom est donné par le fichier chemin_vers_datalist, les transforme en numpy array et les mets dans une liste
    Parameters:
        chemin_vers_images (string) : chemin vers les images d'entrainement du modèle
        chemin_vers_datalist (string) : chemin vers les fichiers avec les noms des images avec lesquels le modèle doit être entrainé
        largeur (int) : taille en largeur souhaité pour l'image
        hauteur (int) : taille en hauteur souhaité pour l'image
    Return:
        liste_image_pixel (list) : liste de numpy array, chaque numpy array étant une image
    """
 
    dataset_images = pds.read_csv(chemin_vers_datalist, sep="\t",header=None,low_memory=False)
    liste_nom_image = dataset_images.values.tolist()
    liste_image_pixel = []
    nombre_images=len(liste_nom_image)
    for i in range(0 , nombre_images):
        nom=str(liste_nom_image[i])
        nom=nom[2:len(nom)-2]
        image = Image.open(f'{chemin_vers_images}/{nom}')
        image = img.resize((largeur, hauteur))
        image = np.array(image)
        liste_image_pixel.append(image)
    
    liste_image_pixel = np.array(liste_image_pixel)
    liste_image_pixel = liste_image_pixel.astype('float32') / 255.0 # il faut normaliser pour + de puissance
    
    return liste_image_pixel

In [ ]:
def encodeur_decodeur(largeur, hauteur):
    """Création de l'autoencodeur qui permettra d'entrainer le modèle, de l'encodeur et du décodeur
    Parameters:
        largeur (int) : largeur des images
        hauteur (int) : hauteur des images
    Return:
        autoencoder (keras.model) : autoencodeur permettant d'entrainer le modèle
        encoder (keras.model) : encodeur permettant d'encoder les images
        decoder (keras.model) : decodeur permettant de decoder les images
    """

    input_shape = (largeur, hauteur, 3) #taille des input
    dropout_level = 0.2 #Dropout level
    
    # Couche de l'encodeur
    input_img = Input(shape=input_shape)
    x = keras.layers.Conv2D(16, (3, 3),strides=1,activation='relu', padding='same')(input_img)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.MaxPooling2D((2, 2), padding='same')(x)
    x = keras.layers.Dropout(dropout_level)(x)
    x = keras.layers.Conv2D(32, (3, 3),strides=1,activation='relu', padding='same')(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.MaxPooling2D((2, 2), padding='same')(x)
    x = keras.layers.Dropout(dropout_level)(x)
    x = keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.MaxPooling2D((2, 2), padding='same')(x)
    x = keras.layers.Dropout(dropout_level)(x)
    x = keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = keras.layers.MaxPooling2D((2, 2), padding='same')(x)
    x = keras.layers.Dropout(dropout_level)(x)
    x = keras.layers.Flatten()(x)
    encoded = keras.layers.Dense(512,activation='relu')(x)


    # Couche du décodeur
    x = keras.layers.Dense(512,,"relu")(encoded)
    x = keras.layers.Dense(8*8*64,"relu")(x)
    x = keras.layers.Reshape((8, 8, 64))(x)
    x = keras.layers.Conv2DTranspose(64,(3,3), activation='relu', padding='same')(x)
    x = keras.layers.UpSampling2D((2, 2))(x)
    x = keras.layers.Dropout(dropout_level)(x)
    x = keras.layers.Conv2DTranspose(64, (3, 3), activation='relu', padding='same')(x)
    x = keras.layers.UpSampling2D((2, 2))(x)
    x = keras.layers.Dropout(dropout_level)(x)
    x = keras.layers.Conv2DTranspose(32, (3, 3), activation='relu', padding='same')(x)
    x = keras.layers.UpSampling2D((2, 2))(x)
    x = keras.layers.Dropout(dropout_level)(x)
    x = keras.layers.Conv2D(16, (3, 3), activation='relu', padding='same')(x)
    x = keras.layers.Conv2DTranspose(3, (3, 3), activation='sigmoid', padding='same')(x)
    decoded = keras.layers.UpSampling2D((2, 2))(x)
    
    
    nb_decoded_layers = 15 #à ajouter si on ajoute une couche au décodeur
    
    #Création encoder, autoencoder
    autoencoder = Model(input_img, decoded)
    encoder = Model(input_img, encoded)
    
    #Récupération des couches pour le décoder
    input_encoded_img = keras.Input(shape=(encoded.shape[1:])) #dimension de l'objet encodé
    y = autoencoder.layers[-nb_decoded_layers](input_encoded_img)
    
    for i in range(nb_decoded_layers-1,0,-1): # 4 3 2 1
        y = autoencoder.layers[-i](y)
    
    # Decoder
    decoder = Model(input_encoded_img, y)
    
    # On compile les modèles
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.0005)
    autoencoder.compile(optimizer=optimizer, loss='MSE', metrics=['accuracy'])
    encoder.compile(optimizer=optimizer, loss='MSE', metrics=['accuracy'])
    decoder.compile(optimizer=optimizer, loss='MSE', metrics=['accuracy'])
    
    return autoencoder,encoder,decoder

In [ ]:
def plot_loss(autoencoder):
    """ Plot la loss de l'auto encodeur : si on a un bon modèle, val loss est proche de loss, et on veut une loss faible
    Parameters:
        autoencoder (keras.model) : autoencodeur contenant l'historique de la loss
    """

    history = autoencoder.history.history

    plt.plot(history['val_loss'],label="test")
    plt.plot(history['loss'],label="training")
    plt.xlabel("epochs")
    plt.ylabel("Loss")
    plt.legend()
    plt.show()

In [ ]:
def plot_image_reconstruction(autoencoder,liste_images, largeur, hauteur,n=10):
    """Affiche l'image original et l'image encoder et decoder par notre autoencodeur de manière à suivre les améliorations du modèle
    Parameters:
        autoencoder (keras.model) : autoencodeur permettant d'encoder et de decoder les images
        liste_images (list) : liste contenant les images sous forme de numpy array 
        largeur (int) : largeur des images
        hauteur (int) : hauteur des images
        n (int) : nombre d'images que l'on souhaite afficher, 10 par défault
    """
    
    image_decode = autoencoder.predict(liste_images)

    plt.figure(figsize=(20, 4))
    for i in range(n):
        # Affiche l'image original
        ax = plt.subplot(2, n, i + 1)
        plt.imshow(liste_images[i].reshape(largeur,hauteur,3))
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)

        # Affiche l'image reconstruite
        ax = plt.subplot(2, n, i + 1 + n)
        plt.imshow(image_decode[i].reshape(largeur, hauteur,3))
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
    plt.show()

In [ ]:
autoencoder,encoder,decoder = encodeur_decodeur(128, 128)
autoencoder.summary()
encoder.summary()
decoder.summary()

L'entrainement étant assez long la fonction *entrainement_modele* permet d'entrainer le modèle en plusieurs fois en veillant à ce qu'ils soient enregistrer à chaque fin d'entrainement dans le cas où le programme est interrompus.

In [ ]:
def entrainement_modeles(autoencoder, encoder, decoder, largeur, hauteur, chemin_modele, nom_modele, chemin_dataset, numero_dataset_debut, numero_dataset_fin ):
    """Permet d'entrainer le modèle sur un nombre de dataset donnée. A chaque fois que le modèle est entrainer sur un dataset il est sauvegarder afin de le conserver dans le cas où l'ordinateur s'arrete.
    Parameters:
        autoencoder (keras.model) : autoencodeur permettant d'encoder et de decoder les images
        encoder (keras.model) : encodeur permettant d'encoder les images
        decoder (keras.model) : decoder permettant de decoder les images
        largeur (int) : largeur des images
        hauteur (int) : hauteur des images
        chemin_model (string) : chemin où l'on souhaite enregistrer le modèle
        nom_model (string) : nom que l'on souhaite donner au modèle
        chemin_dataset (string) : chemin où les datasets sont enregistré
        numero_dataset_debut (int) : numero du dataset où l'on souhaite commencer l'entrainement
        numero_dataset_fin (int) : numero du dataset où l'on souhaite arreter l'entrainement
    """
    for num_dataset in range(numero_dataset_debut, numero_dataset_fin): 
        image_set= chemin_dataset+"/dataset"+str(num_dataset)+".txt"
        liste_image_pixel= load_dataset('img_align_celeba', image_set, largeur, hauteur)

        X_train, X_test = train_test_split(liste_image_pixel,
                                           test_size=0.2, 
                                           random_state=0)
        del(liste_image_pixel) #On supprime la liste une fois qu'elle a été séparé en deux pour libérer de la mémoire
        autoencoder.fit(X_train, X_train,
                        epochs=100,
                        shuffle=True,
                        validation_data=(X_test, X_test))
        plot_loss(autoencoder)
        plot_image_reconstruction(autoencoder,X_test , largeur, hauteur)#plot le résultat de l'encodeur pas séparé
        del(X_test)
        del(X_train)
        encoder.save(chemin_modele+"/encoder_"+nom_modele)
        decoder.save(chemin_modele+"/decoder_"+nom_modele)
        autoencoder.save(chemin_modele+"/autoencoder_"+nom_modele)

In [ ]:
entrainement_modeles(autoencoder, encoder, decoder, 128, 128, "./Model", "vect_512", "./small_dataset", 0, 35)

Si l'on souhaite reprendre l'entrainement aprèsque le programme est été interrompu:

In [ ]:
autoencoder=load_model("./Model/autoencoder_vect_512")
encoder=load_model("./Model/encoder_vect_512")
decoder=load_model("./Model/decoder_vect_512")

entrainement_modeles(autoencoder, encoder, decoder, 128, 128, "./Model", "vect_512", "./small_dataset", 35, 70)

Nous encodons ensuite 50000 qui seront celle que le client pourra choisir lors de la mise en route du logiciel. Encoder les images au préalable permet d'alleger le contenu du logiciel.

In [ ]:
def charge_images_a_encoder(chemin_vers_images,nombre_image) :
    """Charge les images qui seront ensuite encoder
    Parameters:
        chemin_vers_images (string) : chemin vers les images à encoder
        nombre_image (int) : nombre d'image à encoder
    Return:
        liste_image_pixel (list) : liste de numpy array, chaque numpy array étant une image
    """

    img_list = os.listdir(path_to_data)
    liste_image_pixel = []

    for i in range(nb_im-1000,nb_im):
        img = Image.open(f'{path_to_data}/{img_list[i]}')
        img = img.resize((128, 128))
        img = np.array(img)
        liste_image_pixel.append(img)
        if i%1000==0:
            print(i)
        

    liste_image_pixel = np.array(liste_image_pixel)
    liste_image_pixel = liste_image_pixel.astype('float32') / 255.0 # il faut normaliser pour + de puissance
    #img_pixel_list = img_pixel_list.reshape((len(img_pixel_list), np.prod(img_pixel_list.shape[1:])))  # on met ça en vecteur 1D en ligne, mais pas besoin avec conv2D
    
    return liste_image_pixel

In [ ]:
def save_encoded_img(nombre_image_a_encoder):
    """
        Enregistre les images encodées
    Parameters : 
        nombre_image_a_encoder (int) : nombre d'image à encoder, doit être un multiple de 1000
    """
    encoder=load_model("./Model/encoder_smallset_512_100_8864")
    img_pixel_list= load_images("img_align_celeba",1000)
    encoded_img = encoder.predict(img_pixel_list)
    print(encoded_img.shape)
    nombre_image_a_encoder=nombre_image_a_encoder/1000
    
    for i in range(2,101):
        nb_img= i*1000
        img_pixel_list= load_images("img_align_celeba",nb_img)
        new_encoded_img = encoder.predict(img_pixel_list)
        del(img_pixel_list)
        encoded_img=np.vstack([encoded_img,new_encoded_img])
        del(new_encoded_img)
        print(encoded_img.shape)
        
    np.save(f"Data/{len(encoded_img)}_encoded_img", encoded_img)

In [ ]:
save_encoded_img(nombre_image_a_encoder)